---
layout: global
title: Clustering
displayTitle: 聚类
---

本章节涵盖MLlib中的聚类算法。
[RDD接口的聚类指南](mllib-clustering.html)也包含这些算法的相关信息。

## K均值

[k-means](http://en.wikipedia.org/wiki/K-means_clustering)是一个常用的聚类算法，它将数据点按预定的簇数进行聚集。

K-means算法的基本思想是：以空间中k个点为中心进行聚类，对最靠近他们的对象归类。
通过迭代的方法，逐次更新各聚类中心的值，直至得到最好的聚类结果。

假设要把样本集分为c个类别，算法描述如下：

（1）适当选择c个类的初始中心；

（2）在第k次迭代中，对任意一个样本，求其到c个中心的距离，将该样本归到距离最短的中心所在的类；

（3）利用均值等方法更新该类的中心值；

（4）对于所有的c个聚类中心，如果利用（2）（3）的迭代法更新后，值保持不变，则迭代结束，否则继续迭代。

MLlib实现了一个[k-means++](http://en.wikipedia.org/wiki/K-means%2B%2B)方法的并行版本，叫做[kmeans||](http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf)。

`KMeans`作为一个`Estimator`来实现，产生一个`KMeansModel`作为作为基础模型。

### 输入列

<table class="table">
  <thead>
    <tr>
      <th align="left">参数名</th>
      <th align="left">类型</th>
      <th align="left">默认值</th>
      <th align="left">描述</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>特征向量</td>
    </tr>
  </tbody>
</table>

### 输出列

<table class="table">
  <thead>
    <tr>
      <th align="left">参数名</th>
      <th align="left">类型</th>
      <th align="left">默认值</th>
      <th align="left">描述</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>预测的簇中心点索引</td>
    </tr>
  </tbody>
</table>

**样例**

请参考[Python API文档](api/python/pyspark.ml.html#pyspark.ml.clustering.KMeans)了解更多细节。

完整样例代码可以在[Spark仓库](https://github.com/apache/spark)中的"examples/src/main/python/ml/kmeans_example.py"找到

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

## 隐Dirichlet分配（LDA）

LDA（Latent Dirichlet Allocation）是一种文档主题生成模型，也称为一个三层贝叶斯概率模型，包含词、主题和文档三层结构。
所谓生成模型，就是说，我们认为一篇文章的每个词都是通过“以一定概率选择了某个主题，并从这个主题中以一定概率选择某个词语”这样一个过程得到。
文档到主题服从多项式分布，主题到词服从多项式分布。

LDA是一种非监督机器学习技术，可以用来识别大规模文档集（document collection）或语料库（corpus）中潜藏的主题信息。它采用了词袋（bag of words）的方法，这种方法将每一篇文档视为一个词频向量，从而将文本信息转化为了易于建模的数字信息。但是词袋方法没有考虑词与词之间的顺序，这简化了问题的复杂性，同时也为模型的改进提供了契机。每一篇文档代表了一些主题所构成的一个概率分布，而每一个主题又代表了很多单词所构成的一个概率分布。

`LDA`作为一个`Estimator`来实现，支持`EMLDAOptimizer`和`OnlineLDAOptimizer`，产生一个`LDAModel`作为作为基础模型。
高级用户需要时可以将`EMLDAOptimizer`生成的`LDAModel`转换成`DistributedLDAModel`。

**样例**

请参考[Python API文档](api/python/pyspark.ml.html#pyspark.ml.clustering.LDA)了解更多细节。

完整样例代码可以在[Spark仓库](https://github.com/apache/spark)中的"examples/src/main/python/ml/lda_example.py"找到

In [ ]:
from pyspark.ml.clustering import LDA

# Loads data.
dataset = spark.read.format("libsvm").load("data/mllib/sample_lda_libsvm_data.txt")

# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

## 二分K均值

二分K均值（Bisecting k-means）是一种[层次聚类](https://en.wikipedia.org/wiki/Hierarchical_clustering)算法，使用分割（自顶向下）的逼近：所有的观察值开始是一个簇，递归地向下一个层级分裂。

分裂依据为选择能最大程度降低聚类代价函数（也就是误差平方和）的簇划分为两个簇。
以此进行下去，直到簇的数目等于用户给定的数目k为止。

二分K均值常常比传统K均值算法有更快的计算速度，但产生的簇群与传统K均值算法往往也是不同的。

`BisectingKMeans`作为一个`Estimator`来实现，产生一个`BisectingKMeansModel`作为作为基础模型。

**样例**

请参考[Python API文档](api/python/pyspark.ml.html#pyspark.ml.clustering.BisectingKMeans)了解更多细节。

完整样例代码可以在[Spark仓库](https://github.com/apache/spark)中的"examples/src/main/python/ml/bisecting_k_means_example.py"找到

In [ ]:
from pyspark.ml.clustering import BisectingKMeans

# Loads data.
dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(2).setSeed(1)
model = bkm.fit(dataset)

# Evaluate clustering.
cost = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(cost))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

## 高斯混合模型（GMM）

一个高斯混合模型[Gaussian Mixture Model](http://en.wikipedia.org/wiki/Mixture_model#Multivariate_Gaussian_mixture_model)
表示为数据点服从*k*个子高斯分布的混合分布，每个子分布都有独立的概率。
`spark.ml`实现使用[expectation-maximization](http://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm)算法来推导给定样本下的最大似然。

`GaussianMixture`作为一个`Estimator`来实现，产生一个`GaussianMixtureModel`作为作为基础模型。

### 输入列

<table class="table">
  <thead>
    <tr>
      <th align="left">参数名</th>
      <th align="left">类型</th>
      <th align="left">默认值</th>
      <th align="left">描述</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>特征向量</td>
    </tr>
  </tbody>
</table>

### 输出列

<table class="table">
  <thead>
    <tr>
      <th align="left">参数名</th>
      <th align="left">类型</th>
      <th align="left">默认值</th>
      <th align="left">描述</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>预测的簇中心点索引</td>
    </tr>
    <tr>
      <td>probabilityCol</td>
      <td>Vector</td>
      <td>"probability"</td>
      <td>属于每个簇的概率</td>
    </tr>
  </tbody>
</table>

**样例**

请参考[Python API文档](api/python/pyspark.ml.html#pyspark.ml.clustering.GaussianMixture)了解更多细节。

完整样例代码可以在[Spark仓库](https://github.com/apache/spark)中的"examples/src/main/python/ml/gaussian_mixture_example.py"找到

In [ ]:
from pyspark.ml.clustering import GaussianMixture

# loads data
dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

gmm = GaussianMixture().setK(2).setSeed(538009335)
model = gmm.fit(dataset)

print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)